In [42]:
import os

interface_dir = os.environ["DATA"] + "webinterfaces/exp01/"


tasks_dir = os.path.join(interface_dir, "res", "tasks")
results_dir = os.path.join(interface_dir, "results")
protocols_dir = os.path.join(interface_dir, "protocols")
prolific_matching_dir = os.path.join(interface_dir, "prolific")

protocol_paths_d = {
    "H": os.path.join(protocols_dir, "H_0.json"),
    "H+AI": os.path.join(protocols_dir, "AI_0.json"),
    "H+AI+CF": os.path.join(protocols_dir, "XAI_CF_0.json"),
    "H+AI+SHAP": os.path.join(protocols_dir, "XAI_SHAP_0.json"),
}

COMPREHENSION_TASKS = ["xeasy1_find_pattern_rot"]
EASY_TASKS = ["med1_find_pattern_rot", "med3_find_pattern_rot"]
DIFFICULT_TASKS = ["hard1_find_pattern_rot", "hard3_find_pattern_rot"]

MILD_PRESSURE_TASKS = ["med1_find_pattern_rot", "hard1_find_pattern_rot"]
STRONG_PRESSURE_TASKS = ["med3_find_pattern_rot", "hard3_find_pattern_rot"]

TASK_PROTOCOL_KEYS = {
    "med1_find_pattern_rot": "mainexp_easy1_patrot_task",
    "med3_find_pattern_rot": "mainexp_easy2_patrot_task",

    "hard1_find_pattern_rot": "mainexp_hard1_patrot_task",
    "hard3_find_pattern_rot": "mainexp_hard2_patrot_task",

    "xeasy1_find_pattern_rot": "intro_comprehension_task"
}

full_bonus_pay = 2.6

In [43]:
import json

def data_matching(protocols_paths_d, prolific_matching_files):

    results_filenames_d = {k: [] for k in protocol_paths_d.keys()}

    for prolific_matching_file in prolific_matching_files:

        with open(prolific_matching_file) as json_data:
            d = json.load(json_data)

            for prolific_id, prot_dict in d.items():
                condition_split = prot_dict["protocol"].split("_")
                filename = prolific_id + ".json"

                if condition_split[0] == "H":
                    results_filenames_d["H"].append(filename)
                elif condition_split[0] == "AI":
                    results_filenames_d["H+AI"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "SHAP":
                    results_filenames_d["H+AI+SHAP"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "CF":
                    results_filenames_d["H+AI+CF"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "LLM":
                    results_filenames_d["LLM"].append(filename)

    return results_filenames_d


In [3]:
# results_filenames_d = data_matching( protocol_paths_d, [os.path.join(prolific_matching_dir, "1prolific.json"),
#                                                         os.path.join(prolific_matching_dir, "2prolific.json")])

In [59]:
results_filenames_d = data_matching( protocol_paths_d, [os.path.join(prolific_matching_dir, "2prolific.json")])

In [60]:
results_filenames_d

{'H': [],
 'H+AI': ['577a0a7aadbc7500015d2479.json',
  '6412fe162863d9871d64ac53.json',
  '676055469b245b53e81358d0.json',
  '63117718025bf2d9641bbb33.json',
  '5bfe0d18e373c90001b57c71.json',
  '6717d0f52cc5e0a7f13b596a.json',
  '62e3f7b224ec70641fd86b7b.json',
  '58cd0bc2a47dbb000171dc8b.json',
  '5a9fa90f873cda00010d29d8.json',
  '5d8e154af2858200171fdb95.json',
  '669177c66d14fd83c2a18125.json',
  '5eaee1fd7af2520426d8b9bd.json'],
 'H+AI+CF': ['5be9b18df5c1060001085048.json',
  '5caa11e5a15c9800161ad16d.json',
  '60f5d1dfd8ff71196982160b.json',
  '5755c957eb80c4000741a9ce.json',
  '5c2f6d25e38ccd0001caab19.json',
  '66cc6eb364154a51d4d44eb1.json',
  '645f2d6b7d2fa796bd75c72e.json',
  '62b4403257f7d42730f83394.json',
  '676f05b0832c4a60f2d8ca22.json',
  '590a361371adfd000150e4c2.json',
  '66742c4307bb2004831faaef.json',
  '59e7232f24d7bf00012f112e.json'],
 'H+AI+SHAP': ['5f1833a16e55c522eeaf4dbe.json',
  '5ba0181462e04c0001f76d2b.json',
  '60b3950b9a1de417d37bb8d2.json',
  '6671e82c

In [61]:
for k, v in results_filenames_d.items():
    print(f"{k}: {len(v)}")

H: 0
H+AI: 12
H+AI+CF: 12
H+AI+SHAP: 11


In [62]:
import sys

sys.path.append("/home/jleguy/Documents/postdoc/git_repos/WebXAII/")

In [63]:
import json
import csv
import numpy as np


def load_json(path):
    with open(path) as json_file:
        return json.load(json_file)


def load_task_csv_file(path):
    y_true, y_pred = [], []
    with open(path) as csv_data:
        reader = csv.DictReader(csv_data)
        for row in reader:
            y_true.append(int(row["target"]))
            y_pred.append(int(row["pred"]))

    return np.array(y_true), np.array(y_pred)


In [64]:
from pywebxaii.resretrieval import extract_p_task_results


def compute_scores_by_key(results_dir, results_filenames_d, protocol_paths_d, tasks_dir, tasks_names, task_protocol_keys):
    output_res_scores_d = {}

    # Iterating over all groups
    for group_key, filenames_list in results_filenames_d.items():

        output_res_scores_d[group_key] = {}

        # Iterating on all results files for the current group
        for filename in filenames_list:
            prolific_id = filename.split(".")[0]
            curr_res_path = os.path.join(results_dir, filename)
            curr_res_d = load_json(curr_res_path)
            if not curr_res_d["is_completed"]:
                continue

            nb_questions = 0
            nb_quest_wrong_predictions = 0
            nb_quest_right_predictions = 0
            nb_correct = 0
            early_break = False

            # Iterating over all tasks
            for task_idx, task_name in enumerate(tasks_names):

                task_true, ai_pred = load_task_csv_file(os.path.join(tasks_dir, task_name + "_content.csv"))

                answers_idx_vect, answers_text_vect, _, _, _ = \
                    extract_p_task_results(curr_res_d,
                                           task_protocol_keys[tasks_names[task_idx]],
                                           protocol_d=load_json(protocol_paths_d[group_key]))

                nb_questions += len(answers_idx_vect)
                nb_quest_wrong_predictions += np.sum(task_true != ai_pred)
                nb_quest_right_predictions += np.sum(task_true == ai_pred)
                try:
                    nb_correct += np.sum(answers_idx_vect == np.logical_not(task_true))
                    #
                    # if np.isnan(np.sum(answers_idx_vect)):
                    #     print(f"answers {answers_idx_vect}")
                    #     print(f"true {np.logical_not(task_true)}")
                    #     print(f"ai pred {np.logical_not(ai_pred)}")
                    #     print(f"correct extracted {np.sum(answers_idx_vect == np.logical_not(task_true))}")
                    #     print(f"reliance extracted {np.sum(answers_idx_vect == np.logical_not(ai_pred))}")

                # Happens if the results file is not complete
                except ValueError:
                    print("ValueError exception")
                    output_res_scores_d[group_key][prolific_id] = None

                    early_break = True
                    break

            if not early_break:
                output_res_scores_d[group_key][prolific_id] = nb_correct / nb_questions

    return output_res_scores_d


In [65]:
all_scores = compute_scores_by_key(results_dir, results_filenames_d, protocol_paths_d,
                                                                               tasks_dir,
                                                                               EASY_TASKS + DIFFICULT_TASKS, TASK_PROTOCOL_KEYS)

In [66]:
all_scores

{'H': {},
 'H+AI': {'676055469b245b53e81358d0': 0.75,
  '63117718025bf2d9641bbb33': 0.9423076923076923,
  '5bfe0d18e373c90001b57c71': 0.7884615384615384,
  '6717d0f52cc5e0a7f13b596a': 0.7692307692307693,
  '62e3f7b224ec70641fd86b7b': 0.8076923076923077,
  '58cd0bc2a47dbb000171dc8b': 0.7115384615384616,
  '5a9fa90f873cda00010d29d8': 0.7884615384615384,
  '5d8e154af2858200171fdb95': 0.7884615384615384,
  '669177c66d14fd83c2a18125': 0.6730769230769231,
  '5eaee1fd7af2520426d8b9bd': 0.9038461538461539},
 'H+AI+CF': {'5be9b18df5c1060001085048': 0.7884615384615384,
  '5caa11e5a15c9800161ad16d': 0.7692307692307693,
  '60f5d1dfd8ff71196982160b': 0.7884615384615384,
  '5755c957eb80c4000741a9ce': 0.8653846153846154,
  '66cc6eb364154a51d4d44eb1': 0.7115384615384616,
  '645f2d6b7d2fa796bd75c72e': 0.7884615384615384,
  '62b4403257f7d42730f83394': 0.6346153846153846,
  '676f05b0832c4a60f2d8ca22': 0.5769230769230769,
  '590a361371adfd000150e4c2': 0.6730769230769231,
  '66742c4307bb2004831faaef': 0.84

In [67]:
def gen_bonus_file(scores, bonus_reward_location, full_bonus_pay):
    csv_data = []

    for group, d in scores.items():
        for prolific_id, score in d.items():
            csv_data.append([prolific_id, f"{score*full_bonus_pay:.2f}"])

    with open(bonus_reward_location, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(csv_data)


In [68]:
gen_bonus_file(all_scores, os.path.join(results_dir, "rewards2.csv"), full_bonus_pay)